###The following file shows the serialized version of labeling our pictures

In [ ]:
#here we imported opencv
import cv2
import numpy as np
import glob
WINDOW_SIZE=20
STEP_SIZE=10

####read all images' information

all images' informations have been passed into correspondent place

In [2]:
img_path = '/Users/xdai/colored50/*.jpg'
images = glob.glob(img_path)
img_dict={}

for img_p in images:
    k = img_p[img_p.find('50/',)+3:-4]
    img_dict[k] = cv2.imread(img_p, cv2.IMREAD_COLOR)

In [4]:
#read only_blue color into the variable
only_blue = cv2.imread('/Users/xdai/colored50/only_blue.jpg')
#as the only blue is not pure, we will have to pick the blue pixel out of it
blue_pixel_rgb = only_blue[100][1]

####The picture size

The picture size we used contain 200 * 200 pixels, the following result proves this. Each pixel is a three-element array

In [25]:
print only_blue.shape

(200, 200, 3)


#### Sliding window

We slides our window through the picture, each window has 20 * 20 pixels, and is stored in the colored_set dict.

The colored_set has the format:

keys - location of the windows

Values - the image information

In [237]:
def sliding_window(image, stepSize, windowSize):
    colored_set = {}
    for y in xrange(0, image.shape[0], stepSize):# here we want to make sure all window is included 
        for x in xrange(0, image.shape[1], stepSize):
            colored_set[(x,y)] = image[y:y + windowSize[1], x:x + windowSize[0]]
    return colored_set

In [238]:
small_img_dict = dict()
for k in img_dict.keys():
    img = img_dict[k]
    small_images = sliding_window(img, STEP_SIZE, (WINDOW_SIZE, WINDOW_SIZE)) #windows per image
    small_img_dict[k] = small_images
    

####Format of small_img_dict

In the small_img_dict, it containes all small window in one image. As we have designed, the keys of this dictionary is the "image index", the value of the dictionary is its correspondent information

The values in this dictionary are also dictionarys form, keys of which are location of the window, and values are pixels in that small window. The size of pixels shall be 400

The following code proves that each window contains 20 * 20 pixels

In [239]:
print small_img_dict['1046'][(30,60)].shape

(20, 20, 3)


### Is our window blue or not?

In order to differenciate if the window is blue or not, we first check the 400 pixels in the window. The algorithm is to see if each pixel is blue or not, if blue pixels occupy over 50% of the whole window, we will label it as true, otherwise, false will be labeled

The following blue_pixel_rgb will be used for comparing

In [240]:
print type(blue_pixel_rgb)

<type 'numpy.ndarray'>


In [241]:
#the compute_blue function will take the small window as the variable and process it
#the window contains the 20 * 20 pixels
def compute_blue(window):
    #the counter here will be used to check how many pixels are blue
    counter = 0
    #loop over 400 small pixels
    for y in xrange(20):
        for x in xrange(20):
            if (window[x,y] == blue_pixel_rgb).all():
                counter += 1
    return (counter >= 50)

In [242]:
#here we read all index in the dictionary, and extract it's correspondent value
keys = small_img_dict.keys()
all_image = {}
for image_index in keys:   
    window_blue_dict = {}
    small_image = small_img_dict[image_index] #small_img_dict : k='1026', v='(0,0):[[...]]'
    #here we pass our window's pixels to the compute_blue function and determine if the window is blue/or not
    for y in xrange(0, 200, 20):
        for x in xrange(0, 200, 20):
            window_blue_dict[(x,y)] = compute_blue(small_image[x,y]) # passing 
    all_image[image_index] = window_blue_dict

###Results

All results are stored in all_image dictionary.

Keys-the image index

Values - location and if it's blue

From the resule before, we see, 1046 image contains many blue parts, but most of them are non-blue parts which make sense.

However, we do need to decide the criterias for us to determine if it's blue or not(as of right now, it will be true if the counter > 50)

In [243]:
print all_image['1046']

{(0, 20): False, (20, 20): False, (60, 100): False, (40, 60): False, (20, 0): False, (0, 180): False, (80, 40): False, (100, 180): False, (80, 100): False, (100, 40): False, (180, 160): False, (60, 160): False, (120, 180): True, (120, 80): True, (120, 120): True, (120, 100): True, (0, 80): False, (60, 120): False, (160, 0): False, (100, 80): False, (100, 140): True, (20, 180): False, (80, 180): False, (140, 120): True, (80, 0): False, (180, 40): False, (20, 100): False, (80, 140): True, (160, 40): False, (160, 20): False, (180, 20): False, (80, 60): False, (40, 40): False, (40, 20): False, (140, 100): True, (120, 140): True, (0, 140): False, (140, 0): False, (60, 180): False, (140, 180): False, (100, 120): True, (100, 20): False, (80, 120): False, (40, 0): False, (40, 140): False, (140, 140): True, (80, 80): False, (60, 0): False, (40, 80): False, (160, 160): True, (160, 80): True, (180, 120): False, (60, 60): False, (40, 180): False, (0, 0): False, (80, 20): False, (60, 140): False, (

In [244]:
# all_image['1046'][(140, 60)]
# cv2.imshow("Faces found", image)
x_coord=[]
y_coord=[]
for ((x, y), v) in all_image['1046'].items():
    if v:
        x_coord.append(x)
        y_coord.append(y)
        
xmin = np.min(x_coord)
ymin = np.min(y_coord)
xmax = np.max(x_coord)
ymax = np.max(y_coord)

In [245]:
cv2.imshow("test", img_dict['1046'][ymin:ymax+20, xmin:xmax+20])
cv2.waitKey(0)
cv2.destroyAllWindows()

####Read original images into our file

In order to extract the original images to classify, we import all 49 images into the variable

####TESTING

We used one picture to test if we successfully labeled our objects

In [246]:
original_img = cv2.imread('original_1046.jpg')

In [248]:
cv2.imshow("test", original_img[ymin:ymax+20, xmin:xmax+20])
cv2.waitKey(0)
cv2.destroyAllWindows()

Now, we read all original uncolorred pictures to train the classifier, to detect if the window has the object(bee)

In [254]:
img_path_original = '/Users/xdai/bee_labeled/*.jpg'
images_original = glob.glob(img_path_original)
img_dict_original={}
for img_p in images_original:
    k = img_p[img_p.find('labeled/',)+8:-4]
    img_dict_original[k] = cv2.imread(img_p, cv2.IMREAD_COLOR)

In [255]:
small_ori_img_dict = dict()
for k in img_dict_original.keys():
    img = img_dict_original[k]
    small_images = sliding_window(img, STEP_SIZE, (WINDOW_SIZE, WINDOW_SIZE)) #windows per image
    small_ori_img_dict[k] = small_images

In [256]:
print img_dict_original.keys()

['2861', '2614', '3713', '2964', '4654', '4413', '3490', '114', '3573', '395', '4454', '485', '2524', '4703', '4148', '879', '747', '3478', '3344', '2741', '1372', '2401', '4588', '4216', '438', '1130', '952', '99', '3178', '3524', '4566', '3649', '3313', '4052', '2213', '4156', '1046', '4117', '3012', '4507', '940', '2085', '3194', '3844', '1474', '371', '920', '725', '431']


In [257]:
small_ori_img_dict['952']

{(0, 0): array([[[ 62,  64, 134],
         [ 60,  62, 132],
         [ 63,  67, 138],
         ..., 
         [122, 167, 200],
         [129, 173, 204],
         [112, 157, 184]],
 
        [[ 61,  70, 127],
         [ 45,  56, 113],
         [ 41,  52, 110],
         ..., 
         [ 61, 108, 135],
         [ 66, 114, 138],
         [ 62, 109, 130]],
 
        [[ 88, 112, 140],
         [ 59,  85, 115],
         [ 35,  60,  92],
         ..., 
         [ 61, 116, 129],
         [ 64, 118, 129],
         [ 62, 116, 123]],
 
        ..., 
        [[ 54,  63, 113],
         [ 54,  64, 111],
         [ 46,  59,  97],
         ..., 
         [ 40,  46,  93],
         [ 42,  48,  91],
         [ 46,  54,  91]],
 
        [[ 86, 122, 110],
         [ 94, 130, 118],
         [ 92, 128, 116],
         ..., 
         [ 52,  69,  96],
         [ 61,  79, 102],
         [ 71,  91, 109]],
 
        [[ 92, 142, 102],
         [109, 159, 119],
         [110, 158, 122],
         ..., 
         [ 49, 